# 库

In [1]:
import os, sys, re, json
from time import time, sleep

In [105]:
from vVariableInspector import _getshapeof

In [147]:
from vEs import EsProxy
from vMysql import MysqlProxy, stdSqlData
from vUtil.vLog import frmt
from vUtil.vFile import fprint

In [3]:
import pandas as pd

In [4]:
from vUtil.vTqdm import tqdm, trange, typeOfScript

In [5]:
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

In [6]:
sys.path += [r'C:\\Users\\ict\\LymphV\\程序\\试验品\\Py试验品III\\py225 AminerPeterait']

# 变迁

## sql语句

In [7]:
db = MysqlProxy()

In [245]:
sqlChange = lambda name, aid : f'''
select * from
(
    (
        select {stdSqlData(name)} as 姓名, '论文' as 类型, b.paper_title as 题目, 
        if(not isnull(b.paper_date) and now() > b.paper_date, b.paper_date, str_to_date(concat(b.paper_year, '-1-1'), '%Y-%m-%d')) as 日期, 
        c.display_name as 机构, c.affiliation_id as affId, c.reg_province as 省, c.reg_city as 市 from 
        paper_author_affiliations as a 
        join
        papers as b
        join 
        affiliations as c
        on a.author_id = {stdSqlData(aid)}
        and a.paper_id = b.golaxy_paper_id
        and a.affiliation_id = c. affiliation_id
        and not a.is_deleted
        and not b.is_deleted
        and a.affiliation_id is not null
    ) 
    union
    (
        select {stdSqlData(name)} as 姓名, '专利' as 类型, tb.patent_title as 题目, ifnull(tb.applicant_date, tb.grant_date) as 日期, 
        td.display_name as 机构, td.affiliation_id as affId, td.reg_province as 省, td.reg_city as 市 from 
        patent_authors as ta 
        join
        patent as tb
        join
        patent_applicants as tc
        join 
        affiliations as td
        on ta.author_id = {stdSqlData(aid)}
        and ta.patent_id = tb.golaxy_patent_id
        and ta.patent_id = tc.patent_id
        and tc.applicant_id = td. affiliation_id
        and not ta.is_deleted
        and not tb.is_deleted
        and not tc.is_deleted
        and tc.applicant_id is not null
    )
    union
    (
        select {stdSqlData(name)} as 姓名, '项目' as 类型, jb.project_title as 题目,
        str_to_date(concat(jb.start_year, '-1-1'), '%Y-%m-%d') as 日期,
        jd.display_name as 机构, jd.affiliation_id as affId, jd.reg_province as 省, jd.reg_city as 市 from 
        project_authors as ja
        join
        project as jb
        join 
        affiliations as jd
        on ja.author_id = {stdSqlData(aid)}
        and ja.project_id = jb.golaxy_project_id
        and jb.affiliation_id = jd. affiliation_id
        and not ja.is_deleted
        and not ja.is_deleted
        and jb.affiliation_id is not null
    ) 
) as t
where 日期 is not null
order by 日期
;
'''

## 获取深圳企业

In [9]:
a = db('''select distinct a.id
from temp_db.shenzhen_ent as a
join paper_author_affiliations as b
on a.id = b.affiliation_id''')

In [11]:
b = db('''select distinct a.id
from temp_db.shenzhen_ent as a
join landinn.patent_applicants as b
on a.id = b.applicant_id''')

In [12]:
c = db('''select distinct a.id
from temp_db.shenzhen_ent as a
join landinn.project as b
on a.id = b.affiliation_id''')

In [13]:
len(a), len(b), len(c)

(283, 62588, 45)

In [15]:
(*a['id'], *c['id'])

(283,
 8479,
 8482,
 5112,
 8528,
 8540,
 565,
 857,
 998,
 8893,
 8894,
 3090,
 8915,
 8945,
 8973,
 8516,
 2399,
 4929,
 4248,
 5270,
 2956,
 3924,
 88951092,
 89006525,
 89007015,
 2397,
 88997786,
 3273,
 89016554,
 88996239,
 89016433,
 88996214,
 3186,
 89016677,
 3157,
 1419,
 89011359,
 2975,
 89008225,
 88997128,
 89007839,
 3300,
 89006217,
 88998050,
 89007185,
 89008020,
 88948633,
 89017149,
 88998492,
 88999243,
 88997063,
 88996077,
 88997241,
 88961316,
 1464,
 88997742,
 3202,
 88998437,
 88995888,
 88997646,
 88999142,
 88996013,
 88998526,
 88998610,
 4471,
 88996075,
 88997734,
 88951482,
 88997178,
 88995892,
 88998766,
 88997996,
 88999857,
 89015796,
 449,
 88999232,
 88999502,
 88996706,
 88997176,
 88997243,
 88996739,
 88997437,
 88999846,
 88955551,
 88998397,
 88996460,
 88996721,
 88997298,
 88999712,
 88998687,
 88997673,
 88997878,
 88997281,
 88997713,
 88995920,
 88996903,
 88956180,
 88998621,
 1004,
 88997658,
 88998292,
 88995536,
 88995504,
 8901052

## 获取地址

In [9]:
from driverOps import getDriver

In [41]:
dv = getDriver(0)

In [52]:
re.findall('([^()（）]+)', '深圳市宝安区疾病预防控制中心')

['深圳市宝安区疾病预防控制中心']

In [85]:
lastTime = 0
timeGap = 5
def getLocation (name, location):
    global lastTime
    if location: return location
    while time() < lastTime + timeGap:
        sleep(.5)
    lastTime = time()
    dv.get(f'https://www.tianyancha.com/search?key={name}')
    while not dv.current_url.startswith('https://www.tianyancha.com'):
        sleep(1)
    
    eContents = dv.find_elements_by_xpath('//*[contains(@class,"result")]/*/*/*[@class="content"]')
    
    ### 没查到或者没有地址都是None
    for x in eContents:
        ### 深圳市宝安区疾病预防控制中心（深圳市宝安区卫生检验中心）
        ### 哈尔滨工业大学(深圳)(哈尔滨工业大学深圳科技创新研究院)
        txt = x.find_element_by_xpath('.//*[contains(@class,"name")]').text
        if txt == name or name in re.findall('([^()（）]+)', txt):
            eLocs = x.find_elements_by_xpath('.//*[@class="contact row"]')
            if not eLocs: return None
            location = eLocs[0].text
            if location.startswith('地址：') or location.startswith('地址:'): location = location[3:]
            return location

In [86]:
def getEnterprise ():
    s = '''
        select a.id, a.name, a.valid, b.reg_location as location
        from temp_db.shenzhen_ent as a
        left join landinn.affiliations as b
        on a.id = b.affiliation_id
        where a.location is null and 0 = a.valid
        limit 100;
    '''
    
    while 1:
        rows = db(s)
        if len(rows):
            for i in range(len(rows)): yield rows.loc[i]
        else: break

In [75]:
def getEnterpriseLoc ():
    s = '''
        select a.id, a.name, a.valid, b.reg_location as location
        from temp_db.shenzhen_ent as a
        left join landinn.affiliations as b
        on a.id = b.affiliation_id
        where a.location is null and 0 <= a.valid and b.reg_location is not null;
    '''
    
    while 1:
        rows = db(s)
        if len(rows):
            for i in range(len(rows)): yield rows.loc[i]
        else: break

In [76]:
with tqdm(getEnterpriseLoc()) as tg:
    for eid, name, valid, location in tg:
        frmt(f'{eid} {name}', tqdm=tg)
        location = getLocation(name, location)
        if not location: valid = -1
        else:
            valid += 10
            updateEnterprise(eid, location=location)
        updateEnterprise(eid, valid=valid)

In [70]:
def updateEnterprise (eid, **kw):
    sets = ',\n\t\t'.join(f'{x} = {stdSqlData(y)}' for x, y in kw.items())
    
    s = f'''
        update temp_db.shenzhen_ent
        set {sets}
        where id = {stdSqlData(eid)};
    '''
    return db(s, ifCommit=1)

In [71]:
db.commit()

In [87]:
with tqdm(getEnterprise()) as tg:
    for eid, name, valid, location in tg:
        frmt(f'{eid} {name}', tqdm=tg)
        location = getLocation(name, location)
        if not location: valid = -1
        else:
            valid += 10
            updateEnterprise(eid, location=location)
        updateEnterprise(eid, valid=valid)

KeyboardInterrupt: 

In [88]:
dv.close()

## 判断深圳的区

In [98]:
def getEnterprise ():
    s = '''
        select a.id, a.name, a.valid, a.location
        from temp_db.shenzhen_ent as a
        where a.location is not null and a.district is null and 0 <= valid
        limit 100;
    '''
    
    while 1:
        rows = db(s)
        if len(rows):
            for i in range(len(rows)): yield rows.loc[i]
        else: break

In [99]:
def getDistrict (s):
    s = re.sub('高新.*?区', '南山区', s)
    s = re.sub('前海.*?区', '南山区', s)
    s = re.sub('光明.*?区', '光明区', s)
    s = re.sub('坪山.*?区', '坪山区', s)
    s = re.sub('龙华.*?区', '龙华区', s)
    
    rst = re.findall('(罗湖区|福田区|南山区|宝安区|龙岗区|盐田区|龙华区|坪山区|光明区|大鹏新区)', s)
    return rst[0] if rst else None

In [100]:
with tqdm(getEnterprise()) as tg:
    for eid, name, valid, location in tg:
        frmt(f'{eid} {name}', tqdm=tg)
        
        district = getDistrict(location)
        if district is None: 
            print (eid, name, location)
            updateEnterprise(eid, valid=-valid)
            continue
        updateEnterprise(eid, district=district)

88953918 深圳市中兴通讯股份有限公司上海第二研究所 桂林路396号
88954492 东莞深圳清华大学研究院创新中心 东莞市松山湖园区学府路1号1栋
88960031 深圳楚天机械实业有限公司 宝安县布吉镇上李朗村
88962530 深圳市中金岭南有色金属股份有限公司韶关冶炼厂 韶关市南郊九公里
88996460 深圳流花医院 深圳市春风路
89014081 深圳供电局 深圳市沿河西路2号
89031560 深圳市滨海医院筹备办公室 深圳市莲花路1120号北京大学深圳医院综合楼612室



## 企业-区映射

In [101]:
_e2d = db('''
select id, district from temp_db.shenzhen_ent
where district is not null;
''')

In [103]:
e2d = {_e2d['id'][i] : _e2d['district'][i] for i in range(len(_e2d))}

## 获取深圳学者id

insert into temp_db.shenzhen_scholar
select distinct b.author_id as id
from temp_db.shenzhen_ent as a
join paper_author_affiliations as b
on not b.is_deleted and a.id = b.affiliation_id and a.district is not null and b.author_id is not null
;

replace into temp_db.shenzhen_scholar
select distinct c.author_id as id
from temp_db.shenzhen_ent as a
join patent_applicants as b
join patent_authors as c
on not b.is_deleted and not c.is_deleted 
and a.id = b.applicant_id
and b.patent_id = c.patent_id
and a.district is not null and c.author_id is not null
;

replace into temp_db.shenzhen_scholar
select distinct c.author_id as id
from temp_db.shenzhen_ent as a
join project as b
join project_authors as c
on not b.is_deleted and not c.is_deleted
and a.id = b.affiliation_id 
and b.golaxy_project_id = c.project_id
and a.district is not null and c.author_id is not null
;

## 统计人才迁移的数据结构

In [247]:
class Migration:
    districts = ['罗湖区', '福田区', '南山区', '宝安区', '龙岗区', '盐田区', '龙华区', '坪山区', '光明区', '大鹏新区', 'total']
    modes = {'in' : 1, 'out' : -1, 'total' : 0}
    def __init__ (this):
        this.data = {x : {'total' : {}} for x in this.districts}
    def _move (this, district, city, year, mode):
        data = this.data[district]
        total = this.data['total']
        
        
        ### 先保证dict字段存在，year/city/mode
        for x in [data, total]:
            if year not in x: 
                x[year] = {}
                
        for x in [data[year], data['total'], total[year], total['total']]:
            for y in [city, 'total']:
                if y not in x:
                    x[y] = {x : 0 for x in this.modes}
        
        ### 对该单元和所有total均做自增
        for x in [data, total]:
            for y in [year, 'total']:
                for z in [city, 'total']:
                    x[y][z][mode] += 1
                    x[y][z]['total'] += this.modes[mode]                        
    
    def come (this, district, city, year):
        this._move(district, city, year, 'in')
    def go (this, district, city, year):
        this._move(district, city, year, 'out')
    
    def save (this, path='深圳人才迁移'):
        import os
        import pandas as pd
        if not os.path.exists(path): os.makedirs(path)
        
        fprint(str(this.data), file='originalData.py', path=path, mode='w')
         
        ### 存储四维张量
        ### file
        for district, data in this.data.items():
            writer = pd.ExcelWriter(os.path.join(path, f'{district}.xlsx'))
            years = sorted(x for x in data if x != 'total') + ['total']
            ### sheet
            for year in years:
                if not data[year]: continue
                df = pd.DataFrame(columns=['location big', 'location small', *this.modes])
                citys = [x for x in data[year] if x != 'total'] + ['total']
                ### row & col
                for city in citys:
                    location = city if isinstance(city, tuple) else ('-', city) if city == 'total' else ('深圳市', city)
                    df.loc[len(df)] = [*location, *[data[year][city][x] for x in this.modes]]
                df.to_excel(writer, sheet_name=str(year), index=False)
            writer.save()

## 统计人才迁移

In [248]:
def getScholar ():
    s = '''
        select id from temp_db.shenzhen_scholar
        where %s < id
        limit 2000;
    '''
    
    lastId = -1
    while 1:
        rows = db(s % stdSqlData(lastId))
        if len(rows):
            for i in range(len(rows)): 
                lastId = rows.loc[i][0]
                yield lastId
        else: break

In [249]:
def dealFlow (flow, migration):
    lastPos = None
    lastMode = 0
    for i in range(len(flow)):
        affId = flow['affId'][i]
        city = (flow['省'][i], flow['市'][i])
        year = flow['日期'][i].year

        if affId not in e2d and (flow['省'][i] is None or flow['市'][i] is None): continue
        
        ### 没有判断出区的深圳企业，不知道该怎么办
        if affId not in e2d and flow['市'][i] == '深圳市':
            lastMode = 0
            continue

        if affId in e2d:
            mode = 1
            pos = e2d[affId]
        else:
            mode = -1
            pos = city

        #print (lastPos, pos, lastMode, mode, year)

        if lastMode == 1 and lastPos != pos:
            migration.go(lastPos, pos, year)
        if lastMode and mode == 1 and lastPos != pos:
            migration.come(pos, lastPos, year)
            
        if mode == 1 and lastMode == 1 and lastPos != pos:
            frmt (lastPos, pos, year)

        lastMode = mode
        lastPos = pos

In [257]:
def getMigration ():
    migration = Migration()
    with tqdm(getScholar()) as ts:
        ts.total = db.count('temp_db.shenzhen_scholar').values.item()
        for sid in ts:
            frmt(sid, tqdm=ts)
            flow = db(sqlChange(sid,sid))
            dealFlow(flow, migration)
    return migration

In [258]:
migration = getMigration()

(2021-12-10 04:22:55) 南山区 宝安区 2017
(2021-12-10 04:22:55) 宝安区 南山区 2018
(2021-12-10 04:23:15) 南山区 宝安区 2013
(2021-12-10 04:23:15) 南山区 宝安区 2013
(2021-12-10 04:23:18) 龙华区 南山区 2017
(2021-12-10 04:23:18) 南山区 龙华区 2017
(2021-12-10 04:23:18) 龙华区 南山区 2017
(2021-12-10 04:23:18) 龙华区 南山区 2017
(2021-12-10 04:23:18) 南山区 龙华区 2017
(2021-12-10 04:23:18) 龙华区 南山区 2017
(2021-12-10 04:23:20) 南山区 福田区 2011
(2021-12-10 04:23:25) 南山区 宝安区 2012
(2021-12-10 04:23:25) 宝安区 南山区 2012
(2021-12-10 04:23:25) 南山区 宝安区 2012
(2021-12-10 04:23:25) 宝安区 南山区 2012
(2021-12-10 04:23:25) 南山区 宝安区 2012
(2021-12-10 04:23:25) 宝安区 南山区 2012
(2021-12-10 04:23:33) 南山区 龙岗区 2016
(2021-12-10 04:23:33) 龙岗区 南山区 2016
(2021-12-10 04:23:36) 南山区 宝安区 2016
(2021-12-10 04:23:36) 宝安区 南山区 2016
(2021-12-10 04:23:36) 南山区 宝安区 2016
(2021-12-10 04:23:36) 宝安区 南山区 2016
(2021-12-10 04:23:36) 南山区 宝安区 2016
(2021-12-10 04:23:36) 宝安区 南山区 2016
(2021-12-10 04:23:36) 南山区 宝安区 2016
(2021-12-10 04:23:36) 宝安区 南山区 2017
(2021-12-10 04:24:18) 南山区 龙岗区 2015
(2021-12-10 04:24:18

In [259]:
migration.save()

# 关闭

In [133]:
es.close()

In [45]:
db.close()